In [ ]:
import html5lib 
from bs4 import BeautifulSoup 
import requests
import json 
import asyncio 

In [ ]:
response = requests.get('https://newsroom.pinterest.com/en')
response.content

In [ ]:
soup = BeautifulSoup(response.content, 'html5lib')


In [ ]:
table = soup.find_all('figure',attrs={'class':'image-tile image-tile--image-tile-square'})

In [ ]:
date = []
img_cap = []
for item in table:
    date.append(item.time.string)
    img_cap.append(item.span.string)

In [ ]:
import pandas as pd 
df = pd.DataFrame({
    'date' : date ,
    'caption':img_cap
})

In [ ]:
async def requestData():
    f = df

    for i in range(100):
        date = []
        img_cap = []
        api_df = pd.DataFrame({
            'date': date,
            'caption': img_cap
        })
        res1 = requests.post('https://newsroom.pinterest.com/en/views/ajax?_wrapper_format=drupal_ajax',
                             data={
                                 'view_name': 'posts',
                                 'view_display_id': 'views_posts_homepage_latest_news',
                                 'view_args': 'all',
                                 'view_path': '/node/4031',

                                 'view_dom_id': 'd4f803539419d356b71fa91b7b6be69147a62270963f05e833611b0a3ed20609',
                                 'pager_element': 0,
                                 'page': i,
                                 '_drupal_ajax': 1,
                                 'ajax_page_state[theme]': 'pin_base',

                                 'ajax_page_state[libraries]': ' codesnippet/codesnippet.highlightjs,codesnippet/codesnippet.style.googlecode,footnotes/footnotes,google_analytics/google_analytics,js_sha/js_sha.sha,paragraphs/drupal.paragraphs.unpublished,pin_base/core,pin_captcha/pin_captcha.recaptcha_invisible,pin_fields/pincolorfield,pin_google_analytics/google_analytics_script,pin_newsroom_views_posts/pin_newsroom_views_posts.enable,pin_pinterest_client/client_identification,spamspan/obfuscate,system/base,views/views.ajax,views/views.module'
                             }
                             )
        lst = json.loads(res1.content)
        soup = BeautifulSoup(lst[2]['data'], 'html5lib')
        table = soup.find_all(
            'figure', attrs={'class': 'image-tile image-tile--image-tile-square'})
      

        for item in table:
            date.append(item.time.string)
            img_cap.append(item.span.string)

        f = pd.concat([f, api_df])

    return f


In [ ]:
r = await requestData()
r

In [ ]:
r.shape